# Docling - Advanced Document Processing

`by João Gabriel Lima`

## Description
Docling é um toolkit open-source da IBM Research para conversão avançada de documentos em formatos estruturados para IA Generativa. Desenvolvido especificamente para parsing de PDFs complexos, DOCX, XLSX, HTML e imagens com compreensão de layout, tabelas, código, fórmulas e classificação de imagens. Integração essencial para extração de conteúdo estruturado em projetos de AI.

## Prerequisites
- **Precida de API Key:** Não - biblioteca open source
- **Dependencies:** docling, docling-core, transformers
- **Data Input:** data/ directory, formatos PDF/DOCX/XLSX/HTML/images
- **Last Update:** v2.38.0 (Jun 23, 2025) - 32.7k GitHub stars


In [ ]:
# Installation
!uv add docling docling-core transformers torch pillow pandas tabulate
!uv add "docling[complete]"

## Configuration

In [ ]:
import os
import torch
from pathlib import Path
from docling.document_converter import DocumentConverter
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling_core.types.doc import DoclingDocument
from docling_core.types.doc.document import DocTagsDocument
import json
import pandas as pd

# Configuração de diretórios
DATA_INPUT = Path("../../data/input")
DATA_OUTPUT = Path("../../data/output")
DATA_PROCESSED = Path("../../data/processed")

# Criar diretórios se não existirem
for dir_path in [DATA_INPUT, DATA_OUTPUT, DATA_PROCESSED]:
    dir_path.mkdir(parents=True, exist_ok=True)

print("✅ Docling environment configured successfully!")
print(f"📁 Input directory: {DATA_INPUT.resolve()}")
print(f"📁 Output directory: {DATA_OUTPUT.resolve()}")
print(f"📁 Processed directory: {DATA_PROCESSED.resolve()}")


## Usage Scenarios

### 1. Basic Document Conversion

Conversão básica de documentos usando configurações padrão do Docling.

In [ ]:
def convert_basic_document(source_path: str) -> dict:
    """
    Conversão básica de documento usando Docling
    
    Args:
        source_path: Caminho para o documento a ser convertido
        
    Returns:
        Dict com formatos de saída (markdown, json, html) e documento original
    """
    converter = DocumentConverter()
    result = converter.convert(source_path)
    
    return {
        'markdown': result.document.export_to_markdown(),
        'json': result.document.export_to_json(),
        'html': result.document.export_to_html(),
        'document': result.document
    }

print("✅ Basic document conversion function ready!")


### 2. Advanced Document Processing

Processamento avançado com configurações otimizadas para extração de estruturas complexas como tabelas, imagens e layouts específicos.


In [ ]:
def parse_document_advanced(file_path: str, enable_ocr: bool = True) -> dict:
    """
    Parse avançado de documento com configurações otimizadas
    
    Args:
        file_path: Caminho para o arquivo
        enable_ocr: Se deve ativar OCR para documentos escaneados
        
    Returns:
        Dict com conteúdo estruturado e metadados
    """
    # Configuração otimizada para documentos complexos
    pipeline_options = PdfPipelineOptions()
    pipeline_options.do_ocr = enable_ocr
    pipeline_options.do_table_structure = True
    pipeline_options.table_structure_options.do_cell_matching = True
    pipeline_options.do_picture_extraction = True
    
    converter = DocumentConverter(
        format_options={
            InputFormat.PDF: pipeline_options,
        }
    )
    
    result = converter.convert(file_path)
    
    # Extrair estrutura de seções do documento
    structured_content = extract_document_structure(result.document)
    
    return {
        'success': True,
        'metadata': extract_document_metadata(result.document),
        'content': structured_content,
        'raw_markdown': result.document.export_to_markdown(),
        'raw_json': result.document.export_to_json()
    }

print("✅ Advanced document processing function ready!")


### Document Structure Extraction

Funções para extrair estruturas hierárquicas de documentos acadêmicos e técnicos, identificando seções comuns como abstract, introdução, metodologia, etc.


In [ ]:
def extract_document_structure(document) -> dict:
    """
    Extrair estrutura hierárquica de documentos acadêmicos/técnicos
    
    Args:
        document: DoclingDocument object
        
    Returns:
        Dict com seções estruturadas do documento
    """
    markdown_content = document.export_to_markdown()
    
    sections = {
        'title': '',
        'authors': '',
        'abstract': '',
        'introduction': '',
        'methodology': '',
        'results': '',
        'discussion': '',
        'conclusion': '',
        'references': '',
        'appendix': ''
    }
    
    lines = markdown_content.split('\\n')
    current_section = None
    
    for line in lines:
        line = line.strip()
        
        if line.startswith('# '):
            sections['title'] = line[2:]
        elif any(keyword in line.lower() for keyword in ['## abstract', '## resumo']):
            current_section = 'abstract'
        elif any(keyword in line.lower() for keyword in ['## introduction', '## introdução']):
            current_section = 'introduction'
        elif any(keyword in line.lower() for keyword in ['## methods', '## methodology', '## metodologia']):
            current_section = 'methodology'
        elif any(keyword in line.lower() for keyword in ['## results', '## resultados']):
            current_section = 'results'
        elif any(keyword in line.lower() for keyword in ['## discussion', '## discussão']):
            current_section = 'discussion'
        elif any(keyword in line.lower() for keyword in ['## conclusion', '## conclusão']):
            current_section = 'conclusion'
        elif any(keyword in line.lower() for keyword in ['## references', '## referências']):
            current_section = 'references'
        elif any(keyword in line.lower() for keyword in ['## appendix', '## apêndice']):
            current_section = 'appendix'
        elif current_section and line:
            sections[current_section] += line + '\\n'
    
    return sections

def extract_document_metadata(document) -> dict:
    """
    Extrair metadados abrangentes do documento
    
    Args:
        document: DoclingDocument object
        
    Returns:
        Dict com metadados estruturados
    """
    markdown = document.export_to_markdown()
    
    lines = markdown.split('\\n')
    title = ""
    for line in lines:
        if line.startswith('# '):
            title = line[2:].strip()
            break
    
    return {
        'title': title,
        'word_count': len(markdown.split()),
        'has_tables': '|' in markdown,
        'has_images': '![' in markdown,
        'sections': count_document_sections(markdown),
        'language': detect_document_language(markdown)
    }

def count_document_sections(markdown: str) -> dict:
    """
    Contar seções típicas de documentos acadêmicos/técnicos
    """
    sections = ['abstract', 'introduction', 'methodology', 'results', 'discussion', 'conclusion']
    counts = {}
    
    for section in sections:
        counts[section] = markdown.lower().count(f'## {section}')
    
    return counts

def detect_document_language(markdown: str) -> str:
    """
    Detectar idioma predominante do documento
    """
    # Palavras-chave em português
    pt_keywords = ['resumo', 'introdução', 'metodologia', 'resultados', 'discussão', 'conclusão']
    # Palavras-chave em inglês
    en_keywords = ['abstract', 'introduction', 'methodology', 'results', 'discussion', 'conclusion']
    
    pt_count = sum(1 for keyword in pt_keywords if keyword in markdown.lower())
    en_count = sum(1 for keyword in en_keywords if keyword in markdown.lower())
    
    return 'portuguese' if pt_count > en_count else 'english'

print("✅ Document structure extraction functions ready!")


### 3. SmolDocling VLM Integration

Integração com o modelo SmolDocling para processamento visual de documentos usando Vision Language Models (VLM).


In [ ]:
# SmolDocling VLM - Ultra-compact 256M parameter model
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

def setup_smoldocling():
    """
    Setup SmolDocling VLM model for advanced document understanding
    
    Returns:
        Tuple com (processor, model, device) configurados
    """
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Initialize processor and model
    processor = AutoProcessor.from_pretrained("ds4sd/SmolDocling-256M-preview")
    model = AutoModelForVision2Seq.from_pretrained(
        "ds4sd/SmolDocling-256M-preview",
        torch_dtype=torch.bfloat16,
        _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
    ).to(DEVICE)
    
    return processor, model, DEVICE

print("✅ SmolDocling setup function ready!")


## 3. SmolDocling VLM Integration


## Advanced Features & Capabilities


In [ ]:
# Batch Processing with Performance Optimization
def batch_process_documents_optimized(input_dir: str, output_dir: str, use_vlm: bool = False) -> list:
    """
    Processamento em lote otimizado com múltiplas configurações
    """
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    results = []
    
    # Configuração para diferentes tipos de documento
    configs = {
        'medical': {
            'do_ocr': True,
            'do_table_structure': True,
            'do_picture_extraction': True,
            'table_structure_options.do_cell_matching': True
        },
        'presentation': {
            'do_ocr': False,
            'do_table_structure': True,
            'do_picture_extraction': False,
        }
    }
    
    for file_path in input_path.glob("*"):
        if file_path.suffix.lower() in ['.pdf', '.docx', '.xlsx', '.html', '.png', '.jpg', '.jpeg']:
            print(f"🔄 Processing: {file_path.name}")
            
            try:
                if use_vlm and file_path.suffix.lower() in ['.png', '.jpg', '.jpeg']:
                    # Use SmolDocling for images
                    result = convert_with_smoldocling(str(file_path))
                    content = result['markdown']
                else:
                    # Use standard Docling
                    result = convert_basic_document(str(file_path))
                    content = result['markdown']
                
                # Save result
                output_file = output_path / f"{file_path.stem}_parsed.md"
                with open(output_file, 'w', encoding='utf-8') as f:
                    f.write(content)
                
                results.append({
                    "file": file_path.name,
                    "status": "success",
                    "method": "vlm" if use_vlm else "standard",
                    "output": str(output_file),
                    "size_kb": file_path.stat().st_size // 1024
                })
                
            except Exception as e:
                results.append({
                    "file": file_path.name,
                    "status": "error",
                    "error": str(e)
                })
    
    return results

# Pipeline Integration Class
class DoclingPipeline:
    """
    Pipeline integrado para DocuMed.ai com fallbacks e otimizações
    """
    
    def __init__(self, use_vlm=False, medical_optimized=True):
        self.use_vlm = use_vlm
        self.medical_optimized = medical_optimized
        self.setup_converter()
        
        if use_vlm:
            self.processor, self.model, self.device = setup_smoldocling()
    
    def setup_converter(self):
        """Setup converter with medical optimizations"""
        if self.medical_optimized:
            pipeline_options = PdfPipelineOptions()
            pipeline_options.do_ocr = True
            pipeline_options.do_table_structure = True
            pipeline_options.table_structure_options.do_cell_matching = True
            pipeline_options.do_picture_extraction = True
            
            self.converter = DocumentConverter(
                format_options={
                    InputFormat.PDF: pipeline_options,
                }
            )
        else:
            self.converter = DocumentConverter()
    
    def process_document(self, source_path: str) -> dict:
        """
        Processo principal com fallback automático
        """
        try:
            # Try VLM first if enabled
            if self.use_vlm and Path(source_path).suffix.lower() in ['.png', '.jpg', '.jpeg']:
                return self._process_with_vlm(source_path)
            else:
                return self._process_with_standard(source_path)
                
        except Exception as e:
            # Fallback to basic processing
            print(f"⚠️ Fallback to basic processing: {str(e)}")
            return self._process_basic_fallback(source_path)
    
    def _process_with_vlm(self, source_path: str) -> dict:
        """Process with SmolDocling VLM"""
        return convert_with_smoldocling(source_path)
    
    def _process_with_standard(self, source_path: str) -> dict:
        """Process with standard Docling"""
        result = self.converter.convert(source_path)
        
        return {
            'success': True,
            'document': result.document,
            'markdown': result.document.export_to_markdown(),
            'json': result.document.export_to_json(),
            'metadata': self._extract_metadata(result.document)
        }
    
    def _process_basic_fallback(self, source_path: str) -> dict:
        """Basic fallback processing"""
        converter = DocumentConverter()
        result = converter.convert(source_path)
        
        return {
            'success': True,
            'fallback': True,
            'document': result.document,
            'markdown': result.document.export_to_markdown(),
            'json': result.document.export_to_json()
        }
    
    def _extract_metadata(self, document) -> dict:
        """Extract comprehensive metadata"""
        markdown = document.export_to_markdown()
        
        return {
            'title': self._extract_title(markdown),
            'word_count': len(markdown.split()),
            'has_tables': len(document.tables) > 0,
            'has_images': len(document.pictures) > 0,
            'table_count': len(document.tables),
            'image_count': len(document.pictures),
            'sections': self._count_sections(markdown),
            'confidence': self._estimate_confidence(document)
        }
    
    def _extract_title(self, markdown: str) -> str:
        """Extract document title"""
        lines = markdown.split('\\n')
        for line in lines:
            if line.startswith('# '):
                return line[2:].strip()
        return ""
    
    def _count_sections(self, markdown: str) -> dict:
        """Count document sections"""
        sections = ['abstract', 'introduction', 'methods', 'results', 'discussion', 'conclusion']
        counts = {}
        for section in sections:
            counts[section] = markdown.lower().count(f'## {section}')
        return counts
    
    def _estimate_confidence(self, document) -> float:
        """Estimate processing confidence based on document features"""
        confidence = 0.5  # Base confidence
        
        # Increase confidence for structured elements
        if len(document.tables) > 0:
            confidence += 0.2
        if len(document.pictures) > 0:
            confidence += 0.1
        
        # Decrease confidence for potential issues
        markdown = document.export_to_markdown()
        if len(markdown) < 100:
            confidence -= 0.2
        
        return min(max(confidence, 0.0), 1.0)

print("✅ Advanced pipeline and batch processing ready!")


In [ ]:
# Performance Optimization and Monitoring
def monitor_parsing_performance(results: list) -> dict:
    """
    Monitor and analyze parsing performance metrics
    """
    successful = [r for r in results if r.get('status') == 'success']
    failed = [r for r in results if r.get('status') == 'error']
    
    metrics = {
        'total_documents': len(results),
        'successful_parses': len(successful),
        'failed_parses': len(failed),
        'success_rate': len(successful) / len(results) if results else 0,
        'avg_file_size_kb': sum(r.get('size_kb', 0) for r in successful) / len(successful) if successful else 0,
        'methods_used': {
            'standard': len([r for r in successful if r.get('method') == 'standard']),
            'vlm': len([r for r in successful if r.get('method') == 'vlm'])
        }
    }
    
    return metrics

# Error Handling and Recovery
def robust_document_processing(file_path: str, max_retries: int = 3) -> dict:
    """
    Processamento robusto com múltiplas tentativas e fallbacks
    """
    strategies = [
        lambda: DoclingPipeline(use_vlm=True, medical_optimized=True).process_document(file_path),
        lambda: DoclingPipeline(use_vlm=False, medical_optimized=True).process_document(file_path),
        lambda: DoclingPipeline(use_vlm=False, medical_optimized=False).process_document(file_path),
    ]
    
    for attempt, strategy in enumerate(strategies):
        try:
            result = strategy()
            result['strategy_used'] = attempt + 1
            result['attempts'] = attempt + 1
            return result
        except Exception as e:
            if attempt == len(strategies) - 1:
                return {
                    'success': False,
                    'error': str(e),
                    'attempts': len(strategies),
                    'file': file_path
                }
            continue

# Integration with DocuMed.ai workflow
def integrate_with_documedai_pipeline(source_path: str) -> dict:
    """
    Integração completa com pipeline DocuMed.ai
    """
    # Initialize pipeline
    pipeline = DoclingPipeline(use_vlm=True, medical_optimized=True)
    
    # Process document
    result = pipeline.process_document(source_path)
    
    if result.get('success'):
        # Extract medical-specific structure
        if 'document' in result:
            medical_structure = extract_medical_structure(result['document'])
            result['medical_structure'] = medical_structure
        
        # Save to DocuMed.ai format
        output_path = DATA_PROCESSED / f"{Path(source_path).stem}_documedai.json"
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump({
                'source': source_path,
                'processed_at': pd.Timestamp.now().isoformat(),
                'metadata': result.get('metadata', {}),
                'medical_structure': result.get('medical_structure', {}),
                'markdown': result.get('markdown', ''),
                'confidence': result.get('metadata', {}).get('confidence', 0.5)
            }, f, ensure_ascii=False, indent=2)
        
        result['documedai_output'] = str(output_path)
    
    return result

print("✅ Performance monitoring and robust processing ready!")


## Edge Cases & Limitations


### Known Issues and Limitations
- **Large Files**: Performance degradation with PDFs >100MB - consider splitting documents
- **OCR Language Support**: Best performance with Portuguese/English text
- **Complex Layouts**: Some scientific layouts with overlapping elements may not parse correctly
- **Memory Requirements**: High RAM usage for large documents (recommend 8GB+ for batch processing)
- **SmolDocling Flash Attention**: T4 GPUs require `_attn_implementation=\"eager\"` instead of flash_attention_2
- **Rate Limiting**: No built-in rate limiting for batch processing - implement delays if needed

### Special Considerations
- **Scanned PDFs**: Enable OCR for best results with image-based documents
- **Tables with Merged Cells**: Complex table structures may require manual verification
- **Medical Images**: SmolDocling VLM performs better on charts/diagrams than raw medical imaging
- **Multi-language Documents**: Mixed language documents may have inconsistent extraction quality
- **Handwritten Content**: Limited support for handwritten text in medical documents

### Troubleshooting Guide
1. **Memory Errors**: Reduce batch size or process documents individually
2. **OCR Failures**: Check document image quality and language settings
3. **Table Extraction Issues**: Verify table structure and consider manual preprocessing
4. **VLM Errors**: Fallback to standard Docling processing
5. **Encoding Issues**: Ensure UTF-8 encoding for output files"


## References & Last Update

### Official Documentation
- **Official Docs**: [docling-project.github.io/docling](https://docling-project.github.io/docling/)
- **Repository**: [github.com/docling-project/docling](https://github.com/docling-project/docling)
- **SmolDocling HuggingFace**: [ds4sd/SmolDocling-256M-preview](https://huggingface.co/ds4sd/SmolDocling-256M-preview)
- **Technical Report**: [arXiv:2408.09869](https://arxiv.org/abs/2408.09869)
- **SmolDocling Paper**: [arXiv:2503.11576](https://arxiv.org/abs/2503.11576)
- **IBM Research**: [research.ibm.com/publications/docling](https://research.ibm.com/publications/docling-technical-report)

### Integration Examples
- **LangChain Integration**: [Official Examples](https://docling-project.github.io/docling/examples/rag_with_langchain/)
- **LlamaIndex Integration**: [Official Examples](https://docling-project.github.io/docling/examples/rag_with_llamaindex/)
- **Haystack Integration**: [Official Examples](https://docling-project.github.io/docling/examples/rag_with_haystack/)

### Version Information
- **Last Updated**: June 16, 2025 (v2.37.0)
- **SmolDocling Release**: March 14, 2025
- **GitHub Stars**: 32.4k+ (as of June 2025)
- **License**: MIT License
- **Python Support**: 3.9+
- **Platform Support**: macOS, Linux, Windows (x86_64, arm64)"
